In [82]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import numpy as np
from PIL import Image

print(tf.__version__)
print('GPUs available: {}'.format(len(tf.config.list_physical_devices('GPU'))))

2.2.0
GPUs available: 1


In [83]:
images_dir = 'data/Images/'
out_dir = 'data/UCM.tfrecords'

In [88]:
def open_image(image_loc):
    pixel_data = np.array(Image.open(image_loc).getdata())
    
    width = np.sqrt(pixel_data.shape[0])
    channels = pixel_data.shape[1]

    if width % 1 != 0:
        print('Image {} is not square, returning empty variable'.format(image_loc))
        return None

    
    width = int(width)
    
    return pixel_data.reshape((width, width, channels)) / 255.0 #Normalised

def _int64_feature(value):
    """Wrapper for inserting int64 features into Example proto."""
    if isinstance(value, np.ndarray):
        value = value.flatten().tolist()
    elif not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def _float64_feature(value):
    """Wrapper for inserting float64 features into Example proto."""
    if isinstance(value, np.ndarray):
        value = value.flatten().tolist()
    elif not isinstance(value, list):
        value = [value]
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))


def _bytes_feature(value):
    """Wrapper for inserting bytes features into Example proto."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def ucm_to_tfrecord(directory, out_file):
    writer = tf.io.TFRecordWriter(out_file)
    
    label_dict = {}
    
    for label_int, label in enumerate(os.listdir(directory)):
        label_dict[label_int] = label
        for image_file in os.listdir("{}{}/".format(directory, label)):
            image_loc = "{}{}/{}".format(directory, label, image_file)
            
            image_data = open_image(image_loc)
            
            if image_data is None:
                print("Image {} discarded".format(image_loc))
                continue
            
            height = image_data.shape[0]
            width = image_data.shape[1]
            assert height == width
            
            channels = image_data.shape[2]
            
            
            example = tf.train.Example(features=tf.train.Features(feature={
                'height': _int64_feature(height),
                'width': _int64_feature(width),
                'channels': _int64_feature(channels),
                'label': _int64_feature(label_int),
                'image_raw': _float64_feature(image_data)
            }))
            
            writer.write(example.SerializeToString())
            
    with open('data/label_dict.pkl', 'wb') as f:
        pkl.dump(label_dict, f, pkl.HIGHEST_PROTOCOL)
            
    



In [89]:
ucm_to_tfrecord(images_dir, out_dir)

Image data/Images/intersection/intersection99.tif is not square, returning empty variable
Image data/Images/intersection/intersection99.tif discarded
Image data/Images/storagetanks/storagetanks62.tif is not square, returning empty variable
Image data/Images/storagetanks/storagetanks62.tif discarded
Image data/Images/storagetanks/storagetanks60.tif is not square, returning empty variable
Image data/Images/storagetanks/storagetanks60.tif discarded
Image data/Images/storagetanks/storagetanks63.tif is not square, returning empty variable
Image data/Images/storagetanks/storagetanks63.tif discarded
Image data/Images/airplane/airplane55.tif is not square, returning empty variable
Image data/Images/airplane/airplane55.tif discarded
Image data/Images/airplane/airplane59.tif is not square, returning empty variable
Image data/Images/airplane/airplane59.tif discarded
Image data/Images/airplane/airplane76.tif is not square, returning empty variable
Image data/Images/airplane/airplane76.tif discarde